### 크롤링 실습 3
   + jtbc news 사이트의 **속보 페이지**에서
   + **제목, 기사미리보기, 카테고리, 기자명, 송고날짜**를 추출
   + title, preview, category, reporter, pdate
       * 파일에 저장 : jtbcnews.json  

## 리스트

In [41]:
import requests
from lxml import html

In [42]:
url = 'https://news.jtbc.co.kr/section/list.aspx?scode='
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

In [43]:
def to24h(time):
    # 2022-10-26 오전 11:56:00
    pd = time.split(' ')
    pt = pd[2].split(':')
    
    if (pd[1] == '오후') and (pt[0] != '12'):
        pt[0] = int(pt[0]) + 12
        
    elif (pd[1] == '오전') and (pt[0] == '12'):
        pt[0] = int(pt[0]) - 12
        
    time = f'{pd[0]} {pt[0]}:{pt[1]}:{pt[2]}'
    return time

In [49]:
news = [titles,contents,category,writers,regdate]
titles,contents,category,writers,regdate = [],[],[],[],[]

# for i in range(1,2+1):
# 질의문자열을 위해 params 변수 정의
param = {'pgi': i}
# http header와 질의문자열을 이용해서 requests 모듈 호출
res = requests.get(url, headers=header, params=param)
doctree = html.fromstring(res.text)

docs = doctree.cssselect('dt.title_cr a')
for title in docs:
    tit = title.text_content()
    titles.append(tit)

docs = doctree.cssselect('dd.desc a')
for content in docs:
    cnt = content.text_content().strip().replace('   ','')
    contents.append(cnt)

docs = doctree.cssselect('span.location')
for ctg in docs:
    cate = ctg.text_content().replace('[JTBC','').replace(']','').replace('>','').strip()
    category.append(cate)

docs = doctree.cssselect('span.writer') 
for writer in docs:
    wrt = writer.text_content().strip()
    writers.append(wrt)

docs = doctree.cssselect('span.date') 
for rgd in docs:
    pdate = rgd.text_content().strip()
    regdate.append(to24h(pdate))
print(news)

[['\'변론을\' 이상희 "많이 배웠던 드라마…애정해 주셔서 감사해"', "'스맨파' 엠비셔스의 1위… 원밀리언·어때 충격의 탈락", '미국 캘리포니아서 규모 5.1 지진 "2014년 이후 가장 강력"', "'두 번째 세계' 4라운드 미션 세 곡 오늘(26일) 음원 발매", '한혜진, 정호연과 불화설 마침표 "좋은 사이"', '"핵훈련 하겠다" 통보한 러…\'방사능 폭탄\' 준비 중?', "'결혼에 진심' 윤지성, 스페셜 MC로 출격! 뛰어난 몰입력으로 활약", "'한블리' 신호 착각해 급정거한 앞차 vs 뒤에서 박은 블박차, 과실의 무게는 어느 쪽?", "'세계 다크투어' O.J. 심슨 살인사건 진실 향한 세기의 재판 개정", "'두 번째 세계' 미미&문수아, 각각 김선유&유빈 꺾고 압승!", '박수홍 측 "검찰, 유튜버 김용호 거짓주장 모두 인정 기소"', '"마법 같은 능력과 본능적인 자질"...임윤찬, 글로벌 클래식 매니지먼트사 IMG와 계약', '조상준 국가정보원 기조실장, 사의 표명', '윤, 야당 시정연설 불참에 "30년 헌정사 관행 어제부로 무너져"', '동절기 추가접종 18살 이상 성인 확대…2가 백신 3종 활용'], ["'변론을 시작하겠습니다'  배우 이상희가 '변론을 시작하겠습니다' 종영에 앞서 감사한 마음이 가득 담긴 소감을 26일 전했다.  디즈니+ 오리지널 시리즈 '변론을 시작하겠...", "스맨파  '스트릿 맨 파이터(이하 스맨파)' 결승에 올라간 네 팀이 결정됐다.  25일 방송된 Mnet '스맨파'에서는 세미파이널 미션 '뮤즈 오브 스맨파' 무대가 꾸며졌다.  ...", '〈사진=미국 지질조사국 트위터 캡처〉  미국 지질조사국은 현지시간 25일 오전 11시 42분쯤 캘리포니아주 세븐 트리스 인근 지역에서 5.1 규모의 지진이 발생했다고 밝혔다고 미국 ...', "두 번째 세계  '두 번째 세계' 4라운드에서 미미·문수아·유빈이 선보인 무대의 음원이 공개된다.  JTBC '두 번째 세계' 음원이 26일 정오 각종 음원사이트서 발매

In [46]:
print(len(titles),len(contents),len(category),len(writers),len(regdate))

15 15 15 15 15


## CSV

In [21]:
header = '제목,기사미리보기,카테고리,기자,날짜\t\n'
with open('data/JTBC.csv','a',encoding='utf-8') as f:
    f.write(header)
    for i in range(len(titles)):
        title, content, ctg, writer, rgd = titles[i],contents[i],category[i], writers[i], regdate[i]
        f.write(f'"{title}","{content}","{ctg}","{writer}",{rgd}\n')

IndexError: list index out of range

## JSON

In [17]:
import json
from collections import OrderedDict

In [48]:
# 저장형식 : {'title':, 'writer':,'price':}

JTBC = []
with open('data/JDBC.json','a',encoding='utf-8') as f:
    for i in range(len(titles)):
        new = OrderedDict()
        new['제목'] = titles[i]
        new['기사미리보기'] = contents[i]
        new['카테고리'] = category[i]
        new['기자명'] = writers[i]
        new['송고날짜'] = regdate[i]
        JTBC.append(new)
        
    f.write(json.dumps(JTBC,ensure_ascii=False))